# Building a conversational chatbot using Rasa

## Import statements

In [16]:
import logging, json, warnings
warnings.filterwarnings('ignore')
logging.basicConfig(level='INFO')

import rasa.nlu
import rasa.core
import spacy

## NLU training data

In [47]:
nlu_md = """
## intent:greet
- hey
- hello there
- hi
- yo
- yo what up
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- pce
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- later
- see you later

## intent:affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:deny
- no
- never
- I don't think so
- don't like that
- no way
- not really
- not what I want

## intent: request_flight
- I want to go on a trip
- I want to book a flight
- I want to book a trip
- Need a ticket 
- Help me find a flight
- Get me out of here
- I want to travel
- I want to fly somewhere

## intent: inform
- from [Toronto] (origin)
- I'm coming from [New York] (origin)
- I'm going to [Florida] (destination)
- to [Tokyo] (destination)
- from [aug 10] (depart_date)
- leaving [may 4] (depart_date)
- to [aug 20] (return_date)
- come back [may 18] (return_date)
- budget is [$2000] (budget)
- no more than [1000] (budget)
- max [$400] (budget)
- [100 dollars] (budget)
- 100 - [200] (budget)
"""

%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


## NLU model configuration

In [48]:
config = """
language: "en"

pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "ner_crf"                   # uses the pretrained spacy NER model
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
- name: "ner_synonyms"                # trains the synonyms
""" 
%store config > config.yml

Writing 'config' (str) to file 'config.yml'.


## Train NLU model

In [49]:
from rasa.nlu.training_data import load_data
from rasa.nlu.config import RasaNLUModelConfig
from rasa.nlu.model import Trainer
from rasa.nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa.nlu.utils.spacy_utils:Trying to load spacy model with name 'en'
INFO:rasa.nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-en'.
INFO:rasa.nlu.model:Starting to train component SpacyNLP
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component SpacyTokenizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component CRFEntityExtractor
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component SpacyFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component SklearnIntentClassifier
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.2s finished
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component EntitySynonymMapper
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Successfully saved model into '/Users/gthorani/unit_6/travel_chatbot/models/nlu/current'


## Let's evaluate the NLU model with a sample text

In [51]:
def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("I want to go to Tokyo"))

{
  "intent": {
    "name": "request_flight",
    "confidence": 0.45926091432034116
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "request_flight",
      "confidence": 0.45926091432034116
    },
    {
      "name": "greet",
      "confidence": 0.17895262706040507
    },
    {
      "name": "goodbye",
      "confidence": 0.13514848079259084
    },
    {
      "name": "deny",
      "confidence": 0.10373255905812835
    },
    {
      "name": "affirm",
      "confidence": 0.08649811251486786
    },
    {
      "name": "inform",
      "confidence": 0.036407306253666914
    }
  ],
  "text": "I want to go to Tokyo"
}


## Evaluating NLU model on test data

(Definitely want to use unseen data, but will be using the data we have)

In [52]:
from rasa.nlu.test import run_evaluation

run_evaluation('nlu.md', model_directory)

INFO:rasa.nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-en'.
INFO:rasa.nlu.test:Running model for predictions:
100%|██████████| 60/60 [00:00<00:00, 130.00it/s]
INFO:rasa.nlu.test:Intent evaluation results:
INFO:rasa.nlu.test:Intent Evaluation: Only considering those 60 examples that have a defined intent out of 60 examples
INFO:rasa.nlu.test:F1-Score:  0.86246921182266
INFO:rasa.nlu.test:Precision: 0.8307407407407407
INFO:rasa.nlu.test:Accuracy:  0.9
INFO:rasa.nlu.test:Classification report: 
                precision    recall  f1-score   support

        inform       1.00      1.00      1.00        13
       goodbye       0.87      1.00      0.93        13
         greet       0.87      0.93      0.90        14
request_flight       1.00      1.00      1.00         8
        affirm       0.00      0.00      0.00         5
          deny       0.78      1.00      0.88         7

      accuracy                           0.90        60
     macro avg       0.75      0

{'intent_evaluation': {'predictions': [{'text': 'hey',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.515387292050625},
   {'text': 'hello there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5124383707675445},
   {'text': 'hi',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5211566479904506},
   {'text': 'yo',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5171416639806509},
   {'text': 'yo what up',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.46468845520951985},
   {'text': 'good morning',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.4958152117277596},
   {'text': 'good evening',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.4912563838818412},
   {'text': 'moin',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5037038997310451},
   {'text': 'hey there',
    'intent': 'greet',
    'predicted': 'greet',
  

## Using Rasa core to teach the bot to respond

In [53]:
stories_md = """
## flight path
* request_flight
    - utter_ask_to_where
* inform{'destination':'tokyo'}
    - utter_ask_from_where 
* inform{'place':'toronto'}
    - utter_ask_duration
* inform{'depart_date':'2020-10-01T00:00:00','return_date':'2020-10-15T00:00:00'}
    - utter_budget
* inform{'budget':'1000'}
    - action_flight_offer
    - utter_ok
* affirm
    - goodbye

## flight path 1
* request_flight
* inform{'place':'origin'}
* inform{'place':'destination'}
* inform{'date':'depart_date','date':'return_date'}
* inform{'budget':'1000'}
    - action_flight_offer
    - utter_ok
* affirm
    - goodbye

## say hello
* greet
- utter_greeting

## say goodbye
* goodbye
  - utter_goodbye

## fallback
- utter_unclear

"""

%store stories_md > stories.md

Writing 'stories_md' (str) to file 'stories.md'.


## Define domain

In [58]:
domain_yml = """
--- 
actions: 
  - utter_ask_to_where
  - utter_ask_from_where
  - utter_ask_duration
  - utter_budget
  - utter_ok
  - utter_greeting
  - utter_unclear
  - __main__.ApiAction
entities: 
  - destination
  - origin
  - depart_date
  - return_date
  - budget
intents: 
  - greet
  - goodbye
  - affirm
  - deny
  - request_flight
  - inform
responses: 
  utter_ask_duration: 
    - 
      text: "How long are you going for?"
  utter_ask_to_where: 
    - 
      text: "Where are you leaving from?"
  utter_budget: 
    - 
      text: "What is your budget?"
  utter_greeting: 
    - 
      text: "Hey! How are you?"
  utter_ok: 
    - 
      text: "Is this OK?"
  utter_unclear: 
    - 
      text: "I am not understanding, can you be more clear?"
slots: 
  budget: 
    type: text
  depart_date: 
    type: text
  destination: 
    type: text
  origin: 
    type: text
  return_date: 
    type: text
"""

%store domain_yml > domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


In [72]:
from rasa.core.policies.fallback import FallbackPolicy
from rasa.core.policies.keras_policy import KerasPolicy
from rasa.core.policies.memoization import MemoizationPolicy
from rasa.core.agent import Agent

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.1)

agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), fallback])

# loading our neatly defined training dialogues
training_data = agent.load_data('stories.md')

agent.train(
    training_data,
    validation_split=0.0,
    epochs=200
)

agent.persist('models/dialogue')

INFO:rasa.core.policies.ensemble:MappingPolicy not included in policy ensemble. Default intents 'restart and back will not trigger actions 'action_restart' and 'action_back'.


Exception: Passing policy configuration parameters to `agent.train(...)` is not supported anymore. Specify parameters directly in the policy configuration instead. More info http://legacy-docs.rasa.com/core/migrations.html